In [1]:
import pandas as pd
import numpy as np
from pycaret.regression import setup, compare_models, blend_models,tune_model,predict_model,get_config, finalize_model

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['font.family'] = 'Gulim'

from workalendar.asia import SouthKorea
import pendulum

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [3]:
test

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지
2,2021-01-29,금,2983,270,249,0,294.0,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치
3,2021-02-01,월,2924,108,154,538,322.0,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치
4,2021-02-02,화,2924,62,186,455,314.0,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치
5,2021-02-03,수,2924,59,199,5,286.0,모닝롤/게살모닝샌드 우유/주스 계란후라이/찐계란 소고기죽/흑미밥 시래기된장국 베이컨...,쌀밥/흑미밥/찰현미밥 아욱국 매콤해물볶음 감자조림 미나리나물 포기김치 콥샐러드*렌치D,오므라이스 가쓰오장국 빌소세지구이*구운채소 단감치커리무침 양념고추지 겉절이김치
6,2021-02-04,목,2924,61,211,476,288.0,모닝롤/사과파이 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 아욱국 새송이버섯곤약장...,쌀밥/차조밥/찰현미밥 설렁탕 고등어김치말이찜 볼어묵굴소스볶음 브로콜리숙회*초장 석박...,흑미밥 계란파국 돈육두루치기 감자채파프리카볶음 세발나물오리엔탈무침 포기김치
7,2021-02-05,금,2924,169,252,0,256.0,모닝롤/앙버터모닝빵 우유/주스 계란후라이/찐계란 고구마죽/흑미밥 옹심이국 머위나물무...,쌀밥/흑미밥/찰현미밥 북엇국 닭볶음탕 채소전*장 솎음열무나물무침 포기김치 양상추샐러...,유부초밥/추가밥 온메밀소바 국물떡볶이 순대찜*소금 청경채겉절이 포기김치
8,2021-02-08,월,2924,88,174,690,329.0,모닝롤/스콘 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 꽃게탕 근대나물무침 연두부...,쌀밥/흑미밥/찰현미밥 감자양파국 돈수육*씨앗쌈장 매콤어묵볶음 콩나물파채무침 포기김치...,흑미밥 냉이국 반반치킨 꼬막채소무침 청경채찜 포기김치
9,2021-02-09,화,2924,94,183,542,329.0,모닝롤/치즈팡샌드 우유/주스 계란후라이/찐계란 팥죽/흑미밥 맑은버섯국 시금치나물무침...,쌀밥/기장밥/찰현미밥 장각백숙 적어양념장구이 채소스틱*쌈장 도라지오이초무침 겉절이김...,흑미밥 미역국 매운소불고기 단호박두부탕수 메추리알장조림 석박지


## 전처리
- 일자에서 월과 일을 분리
- 요일을 레이블 인코딩화(EDA로 요일의 중요도 순 파악)
- 월 별, 일 별 중식 석식 수요 차이 파악

In [4]:
train['월'] = pd.DatetimeIndex(train['일자']).month
test['월'] = pd.DatetimeIndex(test['일자']).month
train['주'] = pd.DatetimeIndex(train['일자']).week
test['주'] = pd.DatetimeIndex(test['일자']).week
train['일'] = pd.DatetimeIndex(train['일자']).day
test['일'] = pd.DatetimeIndex(test['일자']).day

train['출근'] = train['본사정원수']-(train['본사휴가자수']+train['본사출장자수']+train['현본사소속재택근무자수'])
train['휴가비율'] = train['본사휴가자수']/train['본사정원수']
train['출장비율'] = train['본사출장자수']/train['본사정원수']
train['야근비율'] = train['본사시간외근무명령서승인건수']/train['출근']
train['재택비율'] = train['현본사소속재택근무자수']/train['본사정원수']

test['출근'] = test['본사정원수']-(test['본사휴가자수']+test['본사출장자수']+test['현본사소속재택근무자수'])
test['휴가비율'] = test['본사휴가자수']/test['본사정원수']
test['출장비율'] = test['본사출장자수']/test['본사정원수']
test['야근비율'] = test['본사시간외근무명령서승인건수']/test['출근']
test['재택비율'] = test['현본사소속재택근무자수']/test['본사정원수']

train['day2']=0
train.loc[train['일']>9, 'day2'] = 1
train.loc[train['일']>19, 'day2'] = 2

test['day2']=0
test.loc[test['일']>9, 'day2'] = 1
test.loc[test['일']>19, 'day2'] = 2

In [5]:
weekday = {
    '월': 1,
    '화': 2,
    '수': 3,
    '목': 4,
    '금': 5
}

train['요일'] = train['요일'].map(weekday)
test['요일'] = test['요일'].map(weekday)

In [6]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,월,주,일,출근,휴가비율,출장비율,야근비율,재택비율,day2
0,2016-02-01,1,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,2,5,1,2401.0,0.019223,0.057670,0.099125,0.0,0
1,2016-02-02,2,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,2,5,2,2378.0,0.019223,0.066513,0.134146,0.0,0
2,2016-02-03,3,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,2,5,3,2365.0,0.021530,0.069204,0.046934,0.0,0
3,2016-02-04,4,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0,2,5,4,2277.0,0.039985,0.084583,0.155907,0.0,0
4,2016-02-05,5,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0,2,5,5,2142.0,0.106882,0.069589,0.015873,0.0,0


### 요일 레이블 인코딩

In [7]:
weekday_rank4dinner = {
    1: 1,
    2: 2,
    3: 5,
    4: 3,
    5: 4,
}

train['요일(석식)'] = train['요일'].map(weekday_rank4dinner)
test['요일(석식)'] = test['요일'].map(weekday_rank4dinner)

train['요일(중식)'] = train['요일']
test['요일(중식)'] = test['요일']

### 월 레이블 인코딩

In [8]:
month_rank4lunch = {
    1: 3,
    2: 1,
    3: 2,
    4: 6,
    5: 7,
    6: 8,
    7: 10,
    8: 9,
    9: 5,
    10: 4,
    11: 11,
    12: 12
}
train['월(중식)'] = train['월'].map(month_rank4lunch)
test['월(중식)'] = test['월'].map(month_rank4lunch)

In [9]:
month_rank4dinner = {
    1: 11,
    2: 2,
    3: 1,
    4: 4,
    5: 8,
    6: 6,
    7: 9,
    8: 7,
    9: 5,
    10: 3,
    11: 10,
    12: 12
}
train['월(석식)'] = train['월'].map(month_rank4dinner)
test['월(석식)'] = test['월'].map(month_rank4dinner)

## 주 EDA

In [10]:
rank = pd.DataFrame(range(1,53))
week_rank_lunch = pd.pivot_table(train,values='중식계',index='주').sort_values(by='중식계').reset_index().drop('중식계',axis=1)
week_rank_dinner = pd.pivot_table(train,values='석식계',index='주').sort_values(by='석식계').reset_index().drop('석식계',axis=1)


week_rank4lunch = {}
for i in range(len(rank)):
    week_rank4lunch[week_rank_lunch['주'][i]] = rank[0][i]


week_rank4dinner = {}
for i in range(len(rank)):
    week_rank4dinner[week_rank_dinner['주'][i]] = rank[0][i]
    
    
train['주(중식)'] = train['주'].map(week_rank4lunch)
test['주(중식)'] = test['주'].map(week_rank4lunch)

train['주(석식)'] = train['주'].map(week_rank4dinner)
test['주(석식)'] = test['주'].map(week_rank4dinner)

## 공휴일 변수 생성

In [11]:
def is_holiday(date):
    holidays = list(map(str, pd.Series(np.array(SouthKorea().holidays(int(date[:4])))[:, 0])))
    
    yesterday = str(np.datetime64(date) - 1)
    tomorrow = str(np.datetime64(date) + 1)

    if tomorrow in holidays and yesterday in holidays:
        return 'S'
    if tomorrow in holidays:
        return 'T'
    elif yesterday in holidays:
        return 'Y'
    else : 
        return 'N'

def week_of_month(x):
    dt = pendulum.parse(x)
    
    wom = dt.week_of_month
    if wom < 0:
        wom += 52
    return wom
    

df = pd.concat([train[['본사정원수', '일자']], test[['본사정원수', '일자']]])
df['년월'] = df['일자'].apply(lambda x : x[:7])
df = df[['년월', '본사정원수']].groupby(by=['년월'], as_index=False).mean()

def member_change(date):
    this_month = date[:7]
    last_month = str(np.datetime64(this_month) - 1)
    
    this_month_member = int(df[df['년월'] == this_month]['본사정원수'])
    last_month_member = int(df[df['년월'] == last_month]['본사정원수'])
    
    
    return  this_month_member - last_month_member

train['공휴일전후'] = train['일자'].apply(is_holiday)
test['공휴일전후'] = test['일자'].apply(is_holiday)

train['몇주차'] = train['일자'].apply(week_of_month)
test['몇주차'] = test['일자'].apply(week_of_month)

## 텍스트(메뉴) 전처리

### train/test에서 중복 메뉴만 변수로서 사용

In [12]:
menu_train = train[['조식메뉴','중식메뉴','석식메뉴']]
menu_test = test[['조식메뉴','중식메뉴','석식메뉴']]
print(menu_train.shape)
print(menu_test.shape)

(1205, 3)
(50, 3)


In [13]:
def clean_split(df):
    df = df.split()
    for j in df:
        if '(' in j or '쌀밥' in df or '김치' in df:
            del df[df.index(j)]    
            
    return df

In [14]:
def make_list_to_string(menu_list):
    menu_str_list = []
    for menu in menu_list:
        if len(menu) == 0:
            continue

        string = " ".join(menu)

        menu_str_list.append(string)
    return menu_str_list

In [15]:
menu_train['중식메뉴_prepro'] = menu_train['중식메뉴'].apply(clean_split)
menu_train['석식메뉴_prepro'] = menu_train['석식메뉴'].apply(clean_split)

menu_test['중식메뉴_prepro'] = menu_test['중식메뉴'].apply(clean_split)
menu_test['석식메뉴_prepro'] = menu_test['석식메뉴'].apply(clean_split)

lunch_train= menu_train.중식메뉴_prepro.tolist()
lunch_test= menu_test.중식메뉴_prepro.tolist()
dinner_train = menu_train.석식메뉴_prepro.tolist()
dinner_test = menu_test.석식메뉴_prepro.tolist()

lunch_list_train  = make_list_to_string(lunch_train)
lunch_list_test   = make_list_to_string(lunch_test)
dinner_list_train = make_list_to_string(dinner_train)
dinner_list_test  = make_list_to_string(dinner_test)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
lunch_matrix_train = vectorizer.fit_transform(lunch_list_train)
lunch_df_train = pd.DataFrame(lunch_matrix_train.toarray(), columns=vectorizer.get_feature_names())

vectorizer = TfidfVectorizer()
dinner_matrix_train = vectorizer.fit_transform(dinner_list_train)
dinner_df_train = pd.DataFrame(dinner_matrix_train.toarray(), columns=vectorizer.get_feature_names())

vectorizer = TfidfVectorizer()
lunch_matrix_test = vectorizer.fit_transform(lunch_list_test)
lunch_df_test = pd.DataFrame(lunch_matrix_test.toarray(), columns=vectorizer.get_feature_names())

vectorizer = TfidfVectorizer()
dinner_matrix_test = vectorizer.fit_transform(dinner_list_test)
dinner_df_test = pd.DataFrame(dinner_matrix_test.toarray(), columns=vectorizer.get_feature_names())

In [17]:
# 중복 메뉴 선정
lunch_intersection = list(set(lunch_df_train.columns) & set(lunch_df_test.columns))
print(len(lunch_intersection))
dinner_intersection = list(set(dinner_df_train.columns) & set(dinner_df_test.columns))
print(len(dinner_intersection))

167
120


## 최종 데이터 셋 구축

In [18]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계', '월', '주', '일',
       '출근', '휴가비율', '출장비율', '야근비율', '재택비율', 'day2', '요일(석식)', '요일(중식)',
       '월(중식)', '월(석식)', '주(중식)', '주(석식)', '공휴일전후', '몇주차'],
      dtype='object')

In [19]:
lunch_df_f = pd.concat([train[['공휴일전후', '몇주차', '요일(중식)','월(중식)','일','주(중식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','중식계']],lunch_df_train[lunch_intersection]],axis=1)
lunch_df_test_f = pd.concat([test[['공휴일전후', '몇주차', '요일(중식)','월(중식)','일','주(중식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수']],lunch_df_test[lunch_intersection]],axis=1)


dinner_df_f = pd.concat([train[['공휴일전후', '몇주차', '요일(석식)','월(석식)','일','주(석식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','석식계']],dinner_df_train[dinner_intersection]],axis=1)
dinner_df_test_f = pd.concat([test[['공휴일전후', '몇주차', '요일(석식)','월(석식)','일','주(석식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수']],dinner_df_test[dinner_intersection]],axis=1)

In [20]:
print(lunch_df_f.shape)
print(lunch_df_test_f.shape)

(1205, 181)
(50, 180)


In [21]:
print(dinner_df_f.shape)
print(dinner_df_test_f.shape)

(1205, 134)
(50, 133)


## OLS

In [22]:
from statsmodels.api import OLS

In [23]:
lunch_df_f['공휴일전후'] = lunch_df_f['공휴일전후'].apply(lambda x: 1 if x=='Y' else 0)

In [24]:
ols = OLS(lunch_df_f['중식계'],lunch_df_f.drop(['중식계'],axis=1))
result = ols.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    중식계   R-squared (uncentered):                   0.988
Model:                            OLS   Adj. R-squared (uncentered):              0.986
Method:                 Least Squares   F-statistic:                              495.3
Date:                Thu, 15 Jul 2021   Prob (F-statistic):                        0.00
Time:                        17:53:14   Log-Likelihood:                         -7236.5
No. Observations:                1205   AIC:                                  1.483e+04
Df Residuals:                    1027   BIC:                                  1.574e+04
Df Model:                         178                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
공휴일전후        126.0471     22.143      5.692      0.000      82.596     169.498
몇주차          -24.1624     11.840     -2.041      0.042     -47.395      -0.930
요일(중식)       -70.3582      4.125    -17.057      0.000     -78.452     -62.264
월(중식)         -1.6483      1.304     -1.264      0.207      -4.207       0.911
일              2.4084      1.739      1.385      0.166      -1.003       5.820
주(중식)          1.4214      0.323      4.405      0.000       0.788       2.055
출근             0.5224      0.014     37.018      0.000       0.495       0.550
휴가비율         519.6207    915.053      0.568      0.570   -1275.966    2315.207
출장비율        1.855e+04   1013.355     18.307      0.000    1.66e+04    2.05e+04
야근비율         288.1807     51.102      5.639      0.000     187.905     388.456
재택비율         933.6960    139.914      6.673      0.000     659.146    1208.246
본사출장자수        -7.3881      0.395    -18.693      0.000      -8.164      -6.613
본사휴가자수        -0.1428      0.326     -0.439      0.661      -0.782       0.496
동태매운탕       -271.9208    176.440     -1.541      0.124    -618.146      74.304
맛살계란말이      -122.2165    132.392     -0.923      0.356    -382.005     137.572
깍두기           31.5976     55.631      0.568      0.570     -77.566     140.761
보리밥          147.9986    375.137      0.395      0.693    -588.125     884.122
딸기요거트d       264.5996    372.064      0.711      0.477    -465.494     994.693
파스타샐러드        65.2823     72.639      0.899      0.369     -77.256     207.820
쇠고기숙주볶음      145.6484     86.923      1.676      0.094     -24.918     316.215
육개장          -50.8887     63.005     -0.808      0.419    -174.521      72.744
바나나          167.5376    123.268      1.359      0.174     -74.349     409.424
수수밥          -11.8990    223.278     -0.053      0.958    -450.032     426.234
떡국            74.7110     83.850      0.891      0.373     -89.825     239.247
겉절이김치        145.2916    148.089      0.981      0.327    -145.301     435.884
아삭이고추된장무침   -136.2340    288.399     -0.472      0.637    -702.153     429.685
도토리묵        -201.2777    182.139     -1.105      0.269    -558.684     156.129
해물누룽지탕       -74.7756     65.233     -1.146      0.252    -202.781      53.230
씨앗쌈장        -182.3683    178.603     -1.021      0.307    -532.837     168.101
비름나물         -12.1154     79.166     -0.153      0.878    -167.461     143.230
순대국밥         170.6274    292.072      0.584      0.559    -402.498     743.753
계란후라이         95.8587    327.603      0.293      0.770    -546.989     738.707
양념간장          51.7997    265.910      0.195      0.846    -469.989     573.589
통들깨부추무침      222.9876    401.113      0.556      0.578    -564.107    1010.082
쌈장            48.7281     58.337      0.835      0.404     -6

In [25]:
vari_df = pd.DataFrame(result.pvalues)
vari_df[vari_df[0]<0.1].index

Index(['공휴일전후', '몇주차', '요일(중식)', '주(중식)', '출근', '출장비율', '야근비율', '재택비율',
       '본사출장자수', '쇠고기숙주볶음', '고등어김치말이찜', '콩나물파채무침', '돈육간장불고기', '오리대패불고기',
       '알타리김치', '귀리밥', '우렁된장찌개', '버섯매운탕', '닭볶음탕', '포기김치', '고등어구이', '삼색유자청무침',
       '소불고기', '오이생채', '석박지', '콩나물불고기', '흑임자d', '미역국', '기장밥', '쌀밥', '청경채찜',
       '사우전d'],
      dtype='object')

In [26]:
ols = OLS(lunch_df_f['중식계'],lunch_df_f[['공휴일전후', '몇주차', '요일(중식)', '주(중식)', '출근', '출장비율', '야근비율', '재택비율',
       '본사출장자수', '콩나물불고기', '동태매운탕', '파인d', '쇠고기숙주볶음', '삼색유자청무침', '소불고기', '바나나',
       '황도d', '콩나물파채무침', '버섯매운탕', '돈육간장불고기', '양상추샐러드', '석박지', '수박', '오리대패불고기',
       '오이생채', '흑임자d', '고등어구이', '닭볶음탕', '청경채찜', '우렁된장찌개', '요거트d','휴가비율','월(중식)','일']])
result = ols.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    중식계   R-squared (uncentered):                   0.987
Model:                            OLS   Adj. R-squared (uncentered):              0.987
Method:                 Least Squares   F-statistic:                              2621.
Date:                Thu, 15 Jul 2021   Prob (F-statistic):                        0.00
Time:                        17:53:15   Log-Likelihood:                         -7308.2
No. Observations:                1205   AIC:                                  1.468e+04
Df Residuals:                    1171   BIC:                                  1.486e+04
Df Model:                          34                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
공휴일전후        116.5849     20.223      5.765      0.000      76.907     156.263
몇주차          -26.6389     10.744     -2.479      0.013     -47.718      -5.560
요일(중식)       -71.5078      3.770    -18.968      0.000     -78.904     -64.111
주(중식)          1.5884      0.297      5.339      0.000       1.005       2.172
출근             0.5313      0.013     42.459      0.000       0.507       0.556
출장비율        1.927e+04    681.810     28.269      0.000    1.79e+04    2.06e+04
야근비율         220.0152     38.759      5.677      0.000     143.971     296.060
재택비율        1073.3216    114.611      9.365      0.000     848.457    1298.187
본사출장자수        -7.7521      0.280    -27.638      0.000      -8.302      -7.202
콩나물불고기       299.8952    139.657      2.147      0.032      25.889     573.902
동태매운탕       -338.5991    153.642     -2.204      0.028    -640.044     -37.154
파인d          208.9824    180.637      1.157      0.248    -145.426     563.391
쇠고기숙주볶음      118.7956     76.396      1.555      0.120     -31.093     268.685
삼색유자청무침      488.9319    210.631      2.321      0.020      75.675     902.189
소불고기         211.5729     60.878      3.475      0.001      92.131     331.015
바나나          221.1655    103.598      2.135      0.033      17.906     424.425
황도d          195.4940    137.450      1.422      0.155     -74.182     465.170
콩나물파채무침      238.9394    115.531      2.068      0.039      12.268     465.610
버섯매운탕       -339.5138    155.522     -2.183      0.029    -644.647     -34.380
돈육간장불고기      148.4678     81.417      1.824      0.068     -11.272     308.208
양상추샐러드       259.1436     72.986      3.551      0.000     115.946     402.341
석박지          143.9949     86.015      1.674      0.094     -24.766     312.755
수박           180.6434     98.840      1.828      0.068     -13.281     374.567
오리대패불고기     -234.9172    139.776     -1.681      0.093    -509.156      39.322
오이생채        -210.6734    100.084     -2.105      0.036    -407.036     -14.310
흑임자d        -406.3212    194.561     -2.088      0.037    -788.049     -24.593
고등어구이       -182.3729     78.079     -2.336      0.020    -335.564     -29.182
닭볶음탕        -197.3561     58.998     -3.345      0.001    -313.110     -81.602
청경채찜         779.4473    278.124      2.803      0.005     233.771    1325.123
우렁된장찌개      -403.5531    182.340     -2.213      0.027    -761.303     -45.803
요거트d         125.3943     88.983      1.409      0.159     -49.189     299.978
휴가비율         103.0875     70.669      1.459      0.145     -35.564     241.739
월(중식)         -0.0226      1.217     -0.019      0.985      -2.411       2.366
일              2.9012      1.586      1.829      0.068      -0.211       6.013
=============================================================

# 중식 예측모델

In [27]:
num_feature = ['요일(중식)', '주(중식)', '출근', '출장비율', '야근비율', '재택비율',
       '본사출장자수', '콩나물불고기', '동태매운탕', '파인d', '쇠고기숙주볶음', '삼색유자청무침', '소불고기', '바나나',
       '황도d', '콩나물파채무침', '버섯매운탕', '돈육간장불고기', '양상추샐러드', '석박지', '수박', '오리대패불고기',
       '오이생채', '흑임자d', '고등어구이', '닭볶음탕', '청경채찜', '우렁된장찌개', '요거트d','휴가비율','중식계']
num_feature.remove('중식계')


lunch_regression_model = setup(data=lunch_df_f[['공휴일전후', '몇주차', '요일(중식)', '주(중식)', '출근', '출장비율', '야근비율', '재택비율',
       '본사출장자수', '콩나물불고기', '동태매운탕', '파인d', '쇠고기숙주볶음', '삼색유자청무침', '소불고기', '바나나',
       '황도d', '콩나물파채무침', '버섯매운탕', '돈육간장불고기', '양상추샐러드', '석박지', '수박', '오리대패불고기',
       '오이생채', '흑임자d', '고등어구이', '닭볶음탕', '청경채찜', '우렁된장찌개', '요거트d','휴가비율','중식계']],
                               target='중식계', train_size=0.8,n_jobs=-1,fold = 8,
                               numeric_features=num_feature,
                              numeric_imputation = 'mean',
                                normalize = True)

,Description,Value
0,session_id,7614
1,Target,중식계
2,Original Data,"(1205, 33)"
3,Missing Values,False
4,Numeric Features,30
5,Categorical Features,2
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(964, 37)"


In [28]:
# 최고 성능 모델 선정(2개)
lunch_regression_best_models = compare_models(fold=8, n_select=2, exclude=['huber','llar','lar','par','lasso'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,67.4981,8167.7292,90.0103,0.8107,0.1123,0.0826,0.8800
gbr,Gradient Boosting Regressor,70.7600,8927.7432,94.1710,0.7927,0.1169,0.0865,0.0488
xgboost,Extreme Gradient Boosting,73.4555,9538.5894,97.2461,0.7790,0.1218,0.0902,0.3362
rf,Random Forest Regressor,73.0200,9792.5499,98.3018,0.7729,0.1235,0.0903,0.1200
lightgbm,Light Gradient Boosting Machine,75.0757,10052.0569,99.6986,0.7680,0.1249,0.0922,0.0700
et,Extra Trees Regressor,75.7148,10396.8620,101.3444,0.7598,0.1276,0.0937,0.0975
ada,AdaBoost Regressor,83.2104,11671.5823,107.7454,0.7284,0.1335,0.1022,0.0325
ridge,Ridge Regression,85.3686,12206.3046,110.2506,0.7171,0.1338,0.1025,0.0050
br,Bayesian Ridge,85.5027,12220.3150,110.2937,0.7169,0.1338,0.1027,0.0062
lr,Linear Regression,85.4414,12236.3914,110.3972,0.7164,0.1339,0.1026,0.8825


In [29]:
lunch_regression_best_models_tuned = [tune_model(i, optimize='MAE') for i in lunch_regression_best_models]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,88.6751,12781.4187,113.0549,0.6571,0.1308,0.1055
1,87.4370,13673.0304,116.9317,0.7085,0.1576,0.1165
2,85.4387,11369.9113,106.6298,0.6876,0.1156,0.0952
3,97.5846,15464.7534,124.3574,0.6304,0.1493,0.1183
4,90.0379,16022.0447,126.5782,0.6932,0.1871,0.1285
5,93.9792,14492.1435,120.3833,0.6789,0.1537,0.1221
6,90.8164,13132.3091,114.5963,0.7049,0.1316,0.1042
7,88.2860,13005.9763,114.0437,0.6873,0.1487,0.1111
Mean,90.2818,13742.6984,117.0719,0.6810,0.1468,0.1127
SD,3.6308,1422.8899,6.0715,0.0242,0.0202,0.0101


In [30]:
lunch_model_f = blend_models(estimator_list=lunch_regression_best_models_tuned, fold=5, optimize='MAE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,72.7446,8538.3582,92.4032,0.7953,0.1152,0.0895
1,73.1623,8493.6175,92.1608,0.7868,0.1065,0.0852
2,80.7881,11673.9864,108.0462,0.7491,0.1469,0.1060
3,74.7012,10062.8739,100.3139,0.7787,0.1337,0.0966
4,72.3258,8432.8319,91.8305,0.8056,0.1146,0.0872
Mean,74.7444,9440.3336,96.9509,0.7831,0.1234,0.0929
SD,3.1267,1272.9220,6.3916,0.0192,0.0148,0.0076


In [31]:
pred = predict_model(lunch_model_f)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,76.4389,9553.2017,97.7405,0.7868,0.1270,0.0975


### 테스트 데이터 예측

In [32]:
submission = pd.read_csv('../data/sample_submission.csv')

In [33]:
final_model_lunch = finalize_model(lunch_model_f)
prep_pipe_lunch = get_config('prep_pipe')
prep_pipe_lunch.steps.append(['trained_model', final_model_lunch])

In [34]:
pred_lunch = prep_pipe_lunch.predict(lunch_df_test_f)

submission.iloc[:,1] = pred_lunch
submission.head()

,일자,중식계,석식계
0,2021-01-27,1056.778148,0
1,2021-01-28,1000.250192,0
2,2021-01-29,733.009836,0
3,2021-02-01,1292.013439,0
4,2021-02-02,1095.676583,0


In [ ]:
`

# 석식 예측모델

## OLS

In [35]:
dinner_df_f['공휴일전후'] = dinner_df_f['공휴일전후'].apply(lambda x: 1 if x=='Y' else 0)

In [37]:
num_feature = ['요일(석식)', '월(석식)', '주(석식)', '출근', '출장비율', '야근비율', '재택비율',
       '본사출장자수', '오므라이스','휴가비율','석식계']
num_feature.remove('석식계')

dinner_regression_model = setup(data=dinner_df_f[['몇주차', '요일(석식)', '월(석식)', '주(석식)', '출근', '출장비율', '야근비율', '재택비율',
       '본사출장자수', '오므라이스','휴가비율','석식계','공휴일전후']], target='석식계', train_size=0.8,n_jobs=-1,fold = 8,
                                                              numeric_features=num_feature,
                                numeric_imputation = 'mean',
#                                 categorical_features = ['일'],
                                normalize = True)

,Description,Value
0,session_id,4575
1,Target,석식계
2,Original Data,"(1205, 13)"
3,Missing Values,True
4,Numeric Features,10
5,Categorical Features,2
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(964, 17)"


In [38]:
dinner_regression_best_models = compare_models(fold=8, n_select=2, exclude=['huber','llar','lar','par','lasso'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,56.7327,7098.6191,83.7595,0.6311,1.0032,0.1216,0.8012
rf,Random Forest Regressor,57.7039,7363.3230,84.8573,0.6161,0.9918,0.1250,0.1050
gbr,Gradient Boosting Regressor,59.8547,7490.9163,86.1255,0.6145,1.0698,0.1240,0.0362
et,Extra Trees Regressor,56.9496,7311.8190,84.8253,0.6130,0.9251,0.1246,0.0788
xgboost,Extreme Gradient Boosting,59.6058,7673.8640,87.1585,0.6019,0.9485,0.1306,0.2788
lightgbm,Light Gradient Boosting Machine,60.5932,7813.6216,87.6957,0.5924,1.0186,0.1280,0.0612
knn,K Neighbors Regressor,64.4745,8561.2191,92.3405,0.5558,1.0130,0.1409,0.0062
br,Bayesian Ridge,71.6537,10547.6201,102.4912,0.4625,1.1646,0.1418,0.0062
ridge,Ridge Regression,71.9460,10564.3353,102.5878,0.4610,1.1632,0.1424,0.0050
lr,Linear Regression,71.9414,10565.3594,102.5926,0.4609,1.1632,0.1424,0.0050


In [39]:
dinner_regression_best_models_tuned = [tune_model(i, optimize='MAE') for i in dinner_regression_best_models]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,48.8501,4172.1246,64.5920,0.7844,0.9190,0.1251
1,52.5497,5942.1861,77.0856,0.7186,1.1940,0.0955
2,62.4679,8664.7751,93.0848,0.4232,0.6921,0.1383
3,67.0286,11155.3296,105.6188,0.4900,0.9987,0.1525
4,53.0111,4817.2808,69.4066,0.7339,0.9217,0.1201
5,57.3937,8130.0053,90.1665,0.6070,1.1190,0.1187
6,58.7943,7972.1836,89.2871,0.4932,0.8901,0.1308
7,59.5948,7200.2342,84.8542,0.7312,1.2871,0.1124
Mean,57.4613,7256.7649,84.2619,0.6227,1.0027,0.1242
SD,5.4888,2111.0742,12.5175,0.1294,0.1781,0.0160


In [40]:
dinner_model_f = blend_models(estimator_list=dinner_regression_best_models, fold=8, optimize='MAE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,51.2435,4792.9423,69.2311,0.7523,0.9150,0.1327
1,51.2452,5603.4937,74.8565,0.7346,1.1832,0.0941
2,62.1770,8036.0857,89.6442,0.4651,0.6977,0.1396
3,64.1546,10346.4895,101.7177,0.5270,1.0181,0.1395
4,50.0865,4665.3250,68.3032,0.7423,0.9115,0.1118
5,56.6959,7903.6294,88.9024,0.6179,1.1056,0.1149
6,56.2313,7728.3195,87.9109,0.5087,0.9128,0.1222
7,58.4048,6700.9277,81.8592,0.7498,1.2513,0.1131
Mean,56.2799,6972.1516,82.8031,0.6372,0.9994,0.1210
SD,4.8805,1801.8997,10.7607,0.1146,0.1667,0.0147


In [41]:
pred = predict_model(dinner_model_f)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,52.7528,5178.4848,71.9617,0.6870,0.6588,0.1136


## 테스트

### 테스트 데이터 예측

In [42]:
final_model_dinner = finalize_model(dinner_model_f)
prep_pipe_dinner = get_config('prep_pipe')
prep_pipe_dinner.steps.append(['trained_model', final_model_dinner])

In [43]:
pred_dinner = prep_pipe_dinner.predict(dinner_df_test_f)
submission.iloc[:,2] = pred_dinner
submission.head()

,일자,중식계,석식계
0,2021-01-27,1056.778148,390.662756
1,2021-01-28,1000.250192,396.998973
2,2021-01-29,733.009836,243.951067
3,2021-02-01,1292.013439,518.587009
4,2021-02-02,1095.676583,476.253476


# 저장

In [44]:
import datetime
today = str(datetime.datetime.now().date()).replace("-","")
print("오늘 날짜 : " + today)

submission.to_csv(f'../submission/{today}_pycaret_(2).csv', index =False)

오늘 날짜 : 20210715
